# Creating and Testing Simulations

In [2]:
import toytree
import ipcoal
import numpy as np
import pandas as pd

In [3]:
import os
HOGTIEDIR = os.path.dirname(os.getcwd())
file1 = os.path.join(HOGTIEDIR, "sampledata", "testmatrix.csv")
file2 = os.path.join(HOGTIEDIR, "sampledata", "testtree.txt")

## Random tree

In [4]:
tree = toytree.rtree.imbtree(ntips=10, treeheight=1e5)
tree.draw(ts='p')

(<toyplot.canvas.Canvas at 0x7f7ee7e27730>,
 <toytree.Render.ToytreeMark at 0x7f7ee7e4a4c0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t317d249eff20425fabd08d292240c1ff"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0 50000 100000

## Simulating Introgression

In [5]:
tree.draw(ts='p', admixture_edges=[(3,8)])

(<toyplot.canvas.Canvas at 0x7f7ee4b4aac0>,
 <toytree.Render.ToytreeMark at 0x7f7ee7e4a9a0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3c9b5cb5b8ce431abb82b0dc94d5cee7"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0 50000 100000

In [14]:
tree.get_tip_labels()

['r8', 'r0', 'r7', 'r1', 'r3', 'r5', 'r4', 'r2', 'r6', 'r9']

In [16]:
mod_introgress =  ipcoal.Model(tree=tree, Ne=1e5, admixture_edges=[(3, 8, 0.6, 0.9)])
mod_introgress.sim_loci(nloci=1, nsites=10000) #1 haploid chromosome
genos_introgress=mod_introgress.write_vcf() #i need a matrix of 1's and 0's
data = genos_introgress.iloc[:, 9:].T

In [17]:
for column in data:
    data[column] = data[column].replace([2,3],1)

In [18]:
data

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
r0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
r1,1,1,0,0,0,0,1,1,0,0,...,0,0,0,1,0,1,1,0,0,0
r2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
r3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
r4,1,1,0,0,0,0,1,1,0,0,...,0,0,0,1,0,1,1,1,0,0
r5,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
r6,0,0,0,0,1,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
r7,1,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
r8,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,1
r9,0,0,1,1,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0


In [19]:
data.to_csv(file1)

## Simulating High ILS

I want to include high ILS as an option for null comparison

In [24]:
mod_highILS =  ipcoal.Model(tree=tree, Ne=1e8, nsamples=1)
mod_highILS.sim_loci(nloci=1, nsites=1000) #1 haploid chromosome
genos_highILS=mod_highILS.write_vcf()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,0,1,.,T,"A,C,G",99,PASS,.,GT,3,1,0,2,0,3,1,1,2,1
1,0,2,.,A,"C,G,T",99,PASS,.,GT,1,0,1,2,3,3,3,3,2,3
2,0,3,.,C,"A,G,T",99,PASS,.,GT,1,2,3,0,1,3,0,3,0,0
3,0,4,.,G,"A,C,T",99,PASS,.,GT,3,1,1,0,1,2,2,1,1,3
4,0,5,.,A,"C,G,T",99,PASS,.,GT,1,0,2,3,1,2,2,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,996,.,T,"A,C,G",99,PASS,.,GT,0,2,1,0,1,1,2,1,3,0
996,0,997,.,G,"A,C,T",99,PASS,.,GT,1,1,0,0,2,0,1,1,3,0
997,0,998,.,G,"A,C,T",99,PASS,.,GT,0,2,2,3,0,2,2,1,2,2
998,0,999,.,A,"C,G,T",99,PASS,.,GT,0,3,1,3,1,3,2,3,1,2


In [25]:
data_highILS = genos_highILS.iloc[:, 9:].T
data_highILS

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
r0,3,1,1,3,1,2,2,0,1,1,...,2,3,3,0,0,0,1,0,0,0
r1,1,0,2,1,0,0,0,1,2,3,...,1,2,0,2,2,2,1,2,3,0
r2,0,1,3,1,2,0,0,1,3,1,...,3,1,0,2,3,1,0,2,1,2
r3,2,2,0,0,3,3,0,0,2,2,...,0,2,3,2,3,0,0,3,3,1
r4,0,3,1,1,1,2,0,0,3,1,...,0,3,2,2,3,1,2,0,1,2
r5,3,3,3,2,2,3,1,0,3,0,...,3,1,1,2,2,1,0,2,3,2
r6,1,3,0,2,2,1,1,0,3,1,...,2,3,1,0,2,2,1,2,2,2
r7,1,3,3,1,1,0,2,0,0,1,...,1,0,1,0,1,1,1,1,3,3
r8,2,2,0,1,2,2,3,2,2,1,...,2,2,0,3,2,3,3,2,1,1
r9,1,3,0,3,3,0,3,0,3,1,...,1,1,2,1,0,0,0,2,2,1


In [4]:
mod_highILS.df

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,2,2,2,0,"((r4:6.71022e+07,(r9:4.8..."
1,0,2,4,2,2,1,"((r4:6.71022e+07,(r9:4.8..."
2,0,4,5,1,1,2,"((r4:6.71022e+07,(r9:4.8..."
3,0,5,6,1,1,3,"((r4:6.71022e+07,(r9:4.8..."
4,0,6,7,1,1,4,"((r4:6.71022e+07,(r9:4.8..."
...,...,...,...,...,...,...,...
594,0,991,995,4,4,594,"(((r9:4.9373e+07,r5:4.93..."
595,0,995,996,1,1,595,"(((r4:6.19382e+06,(r0:85..."
596,0,996,998,2,2,596,"((r3:1.85953e+08,((r9:4...."
597,0,998,999,1,1,597,"(r3:2.33101e+08,((r1:2.1..."


In [5]:
# a dictionary of arguments to style the drawings
kwargs = {
    "ts": "c",
    "tip_labels": True,
    "shared_axis": True,
    "width": 600,
    "height": 200,
    "node_sizes": 6,
}
toytree.mtree(mod_highILS.df.genealogy).draw()

(<toyplot.canvas.Canvas at 0x7fbe9b5902e0>,
  <toytree.Render.ToytreeMark at 0x7fbe9b59fa90>])

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="560.0px" height="200.0px" viewBox="0 0 560.0 200.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7e04f2b86476412f854ced284d4a34d3"> r2 r1 r3 r6 r7 r0 r8 r5 r9 r4 r2 r1 r3 r6 r7 r0 r8 r5 r9 r4 r2 r1 r3 r7 r6 r0 r8 r5 r9 r4 r2 r1 r3 r7 r6 r5 r0 r8 r9 r4

In [9]:
genealogies = np.array(mod_highILS.df['genealogy'])
unique_genos = np.unique(genealogies)
len(unique_genos)

594

In [12]:
genealogies

array(['((r4:6.71022e+07,(r9:4.8968e+07,(r5:1.86648e+07,r8:1.86648e+07):3.03032e+07):1.81342e+07):1.49823e+08,((r0:2.58075e+07,r7:2.58075e+07):7.80236e+07,(r6:9.87346e+07,(r3:2.52301e+07,(r1:9.72527e+06,r2:9.72527e+06):1.55048e+07):7.35045e+07):5.0965e+06):1.13094e+08);',
       '((r4:6.71022e+07,(r9:4.8968e+07,(r5:1.86648e+07,r8:1.86648e+07):3.03032e+07):1.81342e+07):1.49823e+08,((r0:2.58075e+07,r7:2.58075e+07):7.80236e+07,(r6:8.50633e+07,(r3:2.52301e+07,(r1:9.72527e+06,r2:9.72527e+06):1.55048e+07):5.98332e+07):1.87678e+07):1.13094e+08);',
       '((r4:6.71022e+07,(r9:4.8968e+07,(r5:1.86648e+07,r8:1.86648e+07):3.03032e+07):1.81342e+07):1.49823e+08,(((r3:2.52301e+07,(r1:9.72527e+06,r2:9.72527e+06):1.55048e+07):5.98332e+07,(r6:3.08275e+07,r7:3.08275e+07):5.42358e+07):1.87678e+07,r0:1.03831e+08):1.13094e+08);',
       '((r4:6.71022e+07,(r9:4.8968e+07,r8:4.8968e+07):1.81342e+07):1.49823e+08,(((r3:2.52301e+07,(r1:9.72527e+06,r2:9.72527e+06):1.55048e+07):5.98332e+07,(r6:3.08275e+07,r7:3.082

In [7]:
trees=[]
for geno in unique_genos:
    tree = toytree.tree(geno, tree_format=0)
    trees.append(tree)

In [10]:
trees

In [22]:
all_tips = []
for tree in trees:
    tips = tree.get_tip_labels()
    all_tips.append(tips)
all_tips = np.array(all_tips)
unique_topos = np.unique(all_tips, axis=0)
len(unique_topos)

347

In [30]:
data_highILS.reindex(all_tips[0])

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
r9,1,3,0,3,3,0,3,0,3,1,...,1,1,2,1,0,0,0,2,2,1
r1,1,0,2,1,0,0,0,1,2,3,...,1,2,0,2,2,2,1,2,3,0
r6,1,3,0,2,2,1,1,0,3,1,...,2,3,1,0,2,2,1,2,2,2
r2,0,1,3,1,2,0,0,1,3,1,...,3,1,0,2,3,1,0,2,1,2
r0,3,1,1,3,1,2,2,0,1,1,...,2,3,3,0,0,0,1,0,0,0
r4,0,3,1,1,1,2,0,0,3,1,...,0,3,2,2,3,1,2,0,1,2
r8,2,2,0,1,2,2,3,2,2,1,...,2,2,0,3,2,3,3,2,1,1
r5,3,3,3,2,2,3,1,0,3,0,...,3,1,1,2,2,1,0,2,3,2
r7,1,3,3,1,1,0,2,0,0,1,...,1,0,1,0,1,1,1,1,3,3
r3,2,2,0,0,3,3,0,0,2,2,...,0,2,3,2,3,0,0,3,3,1


In [50]:
#works
count=0
for list in unique_topos:
    count += 1
    file = os.path.join(HOGTIEDIR, "sampledata", f"genealogy{count}.csv")
    new_df = data_highILS.reindex(list)
    new_df.to_csv(file)

## Slow mutation rate

In [24]:
mod_slowmut =  ipcoal.Model(tree=tree, Ne=1e6, nsamples=1)
mod_slowmut.sim_loci(nloci=1, nsites=100000) #1 haploid chromosome
genos_slowmut=mod_slowmut.write_vcf().iloc[:, 9:].T
genos_slowmut

,0,1,2,3,4,5,6,7,8,9,...,11221,11222,11223,11224,11225,11226,11227,11228,11229,11230
r0,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
r1,1,0,0,0,1,0,1,0,0,0,...,0,1,0,1,0,1,0,1,0,1
r2,1,0,0,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,0
r3,1,0,1,1,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
r4,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,0
r5,1,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,0
r6,1,0,1,1,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
r7,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,1,0,0,1,0,0
r8,1,0,0,0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
r9,1,0,1,1,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [26]:
for col in genos_slowmut:
    genos_slowmut[col] = genos_slowmut[col].replace([2,3],1)

## Trying to simulate SNPs

In [31]:
model = ipcoal.Model(tree, Ne=1e5, nsamples=1) #initialize model

In [32]:
model.sim_snps(nsnps=100) #simulate snps

In [33]:
model.df #look at dataframe

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,1,1,1,0,"(r6:367457,(r8:171024,((..."
1,1,0,1,1,1,0,"((r5:116213,(r8:113972,r..."
2,2,0,1,1,1,0,"(((r0:55561.9,r5:55561.9..."
3,3,0,1,1,1,0,"((r1:176125,(r9:113921,(..."
4,4,0,1,1,1,0,"((r3:141210,(r8:123326,(..."
...,...,...,...,...,...,...,...
95,95,0,1,1,1,0,"((r6:93426,(r2:55068.3,r..."
96,96,0,1,1,1,0,"(((r6:126447,(r4:107894,..."
97,97,0,1,1,1,0,"(r7:190087,((r0:76920.3,..."
98,98,0,1,1,1,0,"((r9:138946,(r3:105167,(..."


In [34]:
kwargs = {
    "ts": "c",
    "tip_labels": True,
    "width": 600,
    "height": 200,
    "node_sizes": 6,
}
toytree.mtree(model.df.genealogy).draw(**kwargs); #draw genealogies

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="200.0px" viewBox="0 0 600.0 200.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta0e0aa404d694f7e94f0ba867caa7ff1"> r4 r1 r7 r5 r9 r3 r0 r2 r8 r6 r6 r3 r4 r1 r2 r0 r7 r9 r8 r5 r7 r4 r9 r3 r2 r8 r6 r1 r5 r0 r2 r0 r7 r8 r6 r3 r5 r4 r9 r1

In [35]:
snps=model.write_vcf()
snp_data = snps.iloc[:, 9:].T
snps

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,0,1,.,C,A,99,PASS,.,GT,0,0,1,0,0,0,0,0,0,0
1,1,1,.,G,T,99,PASS,.,GT,0,0,0,0,0,0,0,0,0,1
2,2,1,.,C,A,99,PASS,.,GT,1,1,0,0,0,1,1,0,1,0
3,3,1,.,T,A,99,PASS,.,GT,1,0,0,0,0,0,0,0,0,0
4,4,1,.,T,G,99,PASS,.,GT,0,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1,.,T,A,99,PASS,.,GT,0,0,1,1,0,0,1,0,0,0
96,96,1,.,T,A,99,PASS,.,GT,0,0,0,0,0,1,0,0,0,0
97,97,1,.,T,C,99,PASS,.,GT,0,0,0,0,0,0,0,0,1,0
98,98,1,.,T,G,99,PASS,.,GT,0,0,0,0,0,0,0,1,0,0


In [45]:
csv = snp_data.to_csv(file1)

In [6]:
mydata = pd.read_csv(file1, index_col=0)
#mydata.drop(["Unnamed: 0"], axis=1)
mydata

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
r0,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
r1,0,0,1,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
r2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
r3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
r4,0,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
r5,0,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
r6,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
r7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
r8,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
r9,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
model.df

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,1,1,1,0,"((r6:130956,(r7:130407,(..."
1,1,0,1,1,1,0,"((r4:125522,(r8:103182,(..."
2,2,0,1,1,1,0,"(r9:356573,(r1:210923,(r..."
3,3,0,1,1,1,0,"((r9:118795,(r0:96411,(r..."
4,4,0,1,1,1,0,"(((r4:62954,(r0:44648.2,..."
...,...,...,...,...,...,...,...
95,95,0,1,1,1,0,"((r9:148718,(r0:97078,r8..."
96,96,0,1,1,1,0,"(r8:968471,((r5:131232,(..."
97,97,0,1,1,1,0,"(r7:265789,(((r0:86569.4..."
98,98,0,1,1,1,0,"((r6:103972,(r8:98686.5,..."


## Simulating SNPs in the Presence of Introgression

I want to simulate an admixture event close to recent between divergent lineages (mimic horizontal gene transfer). I want a simulated dataset that I can run through Hogtie as a test.

In [2]:
hgt_tree = toytree.rtree.imbtree(ntips=10, treeheight=1e10)
hgt_tree.draw(ts='p', admixture_edges=(2,8))

(<toyplot.canvas.Canvas at 0x7ffb1c660820>,
 <toytree.Render.ToytreeMark at 0x7ff9b9760a90>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3821440e3f9c499d84ba3acfce0bc827"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0 5000000000 10000000000

In [3]:
snp_introgression_model =  ipcoal.Model(tree=hgt_tree, Ne=1e8, admixture_edges=[(2, 8, 0.6, 0.9)], nsamples=1)

In [4]:
snp_introgression_model.sim_snps(nsnps=10000, repeat_on_trees=1)

In [5]:
snp_introgression_model.df

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,1,1,1,0,"(r9:1.01568e+10,((r2:1.3..."
1,1,0,1,1,1,0,"(r9:1.01025e+10,((r2:1.4..."
2,2,0,1,1,1,0,"(r9:1.02837e+10,((r2:1.4..."
3,3,0,1,1,1,0,"(r9:1.00285e+10,((r2:1.3..."
4,4,0,1,1,1,0,"(r9:1.00535e+10,((r2:1.6..."
...,...,...,...,...,...,...,...
9995,9995,0,1,1,1,0,"(r9:1.02572e+10,((r2:1.6..."
9996,9996,0,1,1,1,0,"(r9:1.01631e+10,((r2:1.5..."
9997,9997,0,1,1,1,0,"(r9:1.00469e+10,((r2:1.4..."
9998,9998,0,1,1,1,0,"(r9:1.03693e+10,((r2:1.6..."


In [6]:
data = snp_introgression_model.write_vcf().iloc[:, 9:].T

[1, 1, 2, 2, 0, 0, 0, 1, 0, 1]

In [7]:
data

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
r0,1,2,1,3,0,1,2,2,1,1,...,2,1,1,2,0,0,0,2,2,1
r1,1,2,2,3,2,0,0,1,1,2,...,1,0,2,1,2,0,3,1,2,0
r2,2,0,3,2,1,0,1,1,2,3,...,2,0,3,2,2,1,3,1,0,3
r3,2,2,0,3,0,1,0,0,0,3,...,1,2,2,1,1,0,1,2,2,1
r4,0,0,0,0,2,2,0,1,2,1,...,3,2,2,2,3,0,3,0,0,1
r5,0,2,2,2,2,1,1,1,0,3,...,1,2,2,1,0,0,2,1,1,1
r6,0,0,1,2,0,0,3,0,0,1,...,1,0,1,1,1,2,3,3,1,2
r7,1,1,0,1,2,0,3,1,2,3,...,2,1,3,0,2,1,3,2,0,0
r8,0,1,1,1,1,1,2,3,0,1,...,3,1,3,2,2,1,2,0,0,2
r9,1,0,3,2,0,3,0,0,2,0,...,1,2,0,1,1,1,1,1,1,1


In [8]:
for col in data:
    data[col] = data[col].replace([2,3],1)

In [10]:
data

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
r0,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,1
r1,1,1,1,1,1,0,0,1,1,1,...,1,0,1,1,1,0,1,1,1,0
r2,1,0,1,1,1,0,1,1,1,1,...,1,0,1,1,1,1,1,1,0,1
r3,1,1,0,1,0,1,0,0,0,1,...,1,1,1,1,1,0,1,1,1,1
r4,0,0,0,0,1,1,0,1,1,1,...,1,1,1,1,1,0,1,0,0,1
r5,0,1,1,1,1,1,1,1,0,1,...,1,1,1,1,0,0,1,1,1,1
r6,0,0,1,1,0,0,1,0,0,1,...,1,0,1,1,1,1,1,1,1,1
r7,1,1,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,1,0,0
r8,0,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,0,0,1
r9,1,0,1,1,0,1,0,0,1,0,...,1,1,0,1,1,1,1,1,1,1


In [13]:
data.to_csv(file1)

In [3]:
import numpy as np
import pandas as pd

In [15]:
for column in data:
    data[column] = data[column].replace([2,3],1)